# Baker Hughes Worldwide Rig Count vs Oil Prices

This notebook contains comparison of the Baker Hughes Worldwide Rig Count data with spot prices of Brent and WTI oil.

### Data Sources

1. Baker Hughes International Rig Count: available free of charge on [Baker Hughes website](https://rigcount.bakerhughes.com/intl-rig-count/). The specific dataset used here is *Worldwide Rig Counts - Current & Historical Data*
2. Prices of Brent and WTI spot prices - available from [EIA](https://www.eia.gov/dnav/pet/pet_pri_spt_s1_d.htm). Note: both prices are in USD per barrell **spot** FOB prices at Europe (specific location not given in EIA spreadsheet) for Brent and in Cushing, Oklahoma for West Texas Intermediate.

### Sidenote: Importance of Cushing
The importance of Cushing for oil industry is nicely explained in the [video by CME](https://www.cmegroup.com/education/courses/introduction-to-crude-oil/the-importance-of-cushing-oklahoma.html): definitely worth your time! 
Tl;dr: it's the delivery place of oil for NYMEX futures contracts. If you want to find out even more I strongly recommend Iain Clark's book [*Commodity Option Pricing: A Practitioner's Guide*](https://www.amazon.com/Commodity-Option-Pricing-Practitioners-Finance/dp/1119944511) as an entry point. 
Geography and existing oil infrastructure explains the importance of Cushing - just glance at the map below from a [Business Insider article](https://www.businessinsider.com/cushing-oklahoma-is-the-center-of-the-oil-universe-2016-3?IR=T) - this is clearly not my map, I just share it.

![](./graphics/oil_pipelines_usa.png)

### Sidenote: Delivery Location of Brent Oil

The Brent benchmark is not as simple as its popularity would suggest. For oil market outsider like me, the inner workings of how physical delivery of Brent Oil works seem pretty complex. Suffice it to say that Brent is actually a [basket of various North Sea crudes](https://www.theice.com/brent-crude). 

Concerning the delivery of Brent: it *seems* that the delivery takes place in multiple locations in the North Sea that are agreed upon on a bilateral basis. Much more details are given in this fantastic article: [An Anatomy of the Crude Oil Pricing System](https://www.oxfordenergy.org/wpcms/wp-content/uploads/2011/03/WPM40-AnAnatomyoftheCrudeOilPricingSystem-BassamFattouh-2011.pdf) by Bassam Fattouh.

One more interesting pecularity (feature, actually... that's just the way market evolved - cf. Fattouh's article mentioned above) of the Brent market is the presence of Dated Brent contracts - cf. e.g. [this](https://www.theice.com/products/19118094/Daily-Dated-Brent-Future) ICE site. These contract allow for specifying of the delivery date of crude in the current month (as per [this Wikipedia page](https://en.wikipedia.org/wiki/Brent_Crude#Dated_Brent_assessed_prices_and_the_Brent_spot_market)).

In [48]:
import sys
sys.version

'3.10.4 (main, Jun 14 2022, 14:36:10) [Clang 13.1.6 (clang-1316.0.21.2.5)]'

In [49]:
# import of all packages needed
import pandas as pd
import numpy as np
import requests
import datetime as dt
import os
import re

## 1. Loading data

### 1.1. Baker-Hughes Worldwide Rig Count

In [50]:
# function for data download & processing
def get_rig_oil_count_data(b_load_from_file: bool = True):
    if b_load_from_file:
        str_rig_count_link = os.path.join(os.path.abspath(os.getcwd()), "data", "WorldwideRigCountAug2022.xlsx")
    else:
        str_rig_count_link = "https://rigcount.bakerhughes.com/static-files/e106a3e4-ddd8-4e7d-93a3-01c3de9e7ac0"
    df = pd.read_excel(str_rig_count_link, sheet_name="Worldwide_Rigcount", header=None, skiprows=6,
                       usecols=list(range(1, 11, 1)), na_values="nan")
    # drop redundant rows
    df = df.loc[~(df.loc[:, 1].isin(["Avg."]) | df.loc[:, 1].isna()), :]
    # add columns names
    df.columns = df.loc[0, :]
    # add column with years data
    l_years = [[el]*13 for el in np.arange(2022, 1974, -1)]
    df["which_year"] = [iter_val for iter_list in l_years for iter_val in iter_list]
    # drop rows with locations
    df = df.loc[df["Europe"] != "Europe", :]
    df.rename(columns={2022: "which_month"}, inplace=True)
    # drop further redundant rows
    df = df.loc[~df["Europe"].isna(), :]
    df["which_month"] = [dt.datetime.strptime(el, "%b").month for el in df["which_month"]]

    def make_period(x):
        return pd.Period(dt.date(year=x["which_year"], month=x["which_month"], day=1), freq="M")

    df["which_yearmonth"] = df.apply(func=make_period, axis=1)
    df.drop(columns=["Total Intl.", "which_year", "which_month"], inplace=True)
    df.sort_values("which_yearmonth", ascending=True, inplace=True)
    df.rename(columns={"U.S.": "US"}, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [51]:
df_rigs = get_rig_oil_count_data()
df_rigs.head()

,Latin America,Europe,Africa,Middle East,Asia Pacific,Canada,US,Total World,which_yearmonth
0,327,102,117,160,211,162,1615,2694,1975-01
1,312,110,122,171,211,181,1611,2718,1975-02
2,310,113,122,173,208,192,1651,2769,1975-03
3,303,116,122,169,206,72,1605,2593,1975-04
4,301,124,123,170,202,45,1592,2557,1975-05


In [52]:
df_rigs.rename(
    columns=dict(zip(
        df_rigs.columns, 
        [re.sub(" ", "_", el.lower()) for el in list(df_rigs.columns)])), 
    inplace=True
)

In [53]:
df_rigs.head()

,latin_america,europe,africa,middle_east,asia_pacific,canada,us,total_world,which_yearmonth
0,327,102,117,160,211,162,1615,2694,1975-01
1,312,110,122,171,211,181,1611,2718,1975-02
2,310,113,122,173,208,192,1651,2769,1975-03
3,303,116,122,169,206,72,1605,2593,1975-04
4,301,124,123,170,202,45,1592,2557,1975-05


### 1.2. Brent and WTI Front Month Futures Price Data

In [54]:
def df_load_spot_oil_prices():
    str_oil_prices = os.path.join(os.getcwd(), "data", "oil_prices_eia.csv")
    df = pd.read_csv(str_oil_prices, sep=";")
    df.rename(columns={"brent_spot": "brent", "wti_spot": "wti"}, inplace=True)
    df["quote_date"] = df.apply(func=lambda x: dt.datetime.strptime(x["quote_date"], "%Y-%m-%d").date(), axis=1)
    df = df.loc[df["quote_date"] >= dt.date(year=1991, month=1, day=1), :]
    df = df.sort_values("quote_date", ascending=True)
    df.reset_index(inplace=True, drop=True)
    df[["wti", "brent"]] = df[["wti", "brent"]].fillna(method="ffill")
    df.reset_index(inplace=True, drop=True)
    return df

In [55]:
df = df_load_spot_oil_prices()
df.head()

,quote_date,wti,brent
0,1991-01-02,26.53,26.78
1,1991-01-03,25.61,25.05
2,1991-01-04,24.88,24.08
3,1991-01-07,27.25,25.93
4,1991-01-08,27.50,25.73


In [56]:
df.tail()

,quote_date,wti,brent
8109,2022-09-06,87.35,91.43
8110,2022-09-07,82.50,86.83
8111,2022-09-08,84.04,87.99
8112,2022-09-09,87.27,91.68
8113,2022-09-12,88.18,93.45


In [57]:
# calculate monthly average oil prices
df = df.assign(quote_yearmonth=[pd.Period(el, freq="M") for el in df["quote_date"]])
df_oil = df.groupby("quote_yearmonth")[["wti", "brent"]].mean()
df_oil.reset_index(inplace=True, drop=False)

In [58]:
df_oil.head()

,quote_yearmonth,wti,brent
0,1991-01,25.234091,23.566364
1,1991-02,20.477500,19.536000
2,1991-03,19.901500,19.082500
3,1991-04,20.830000,19.180000
4,1991-05,21.232273,19.186818


### 1.3. Joining the data

## 2. Analysis